In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:

* convolutions need the image data formatted as a cube (width by height by #channels)
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


### CNN with two layers

In [79]:
batch_size = 64
patch_size = 5  # 5x5 patch
depth = 16
num_hidden = 256

graph = tf.Graph()

with graph.as_default():
    
    ## input data 
    ## notice: in CNN, the image will be kept as a square matrix instead of a array
    tf_train_dataset = tf.placeholder(
        tf.float32, shape =(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(
        tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    # First Layer
    
    # CONV 3x3 1
    layer1_weights_11 = tf.Variable(tf.truncated_normal(
        [3, 3, num_channels, depth], stddev = 0.1))
    layer1_biases_11 = tf.Variable(tf.zeros([depth]))
    
    # CONV 5x5 1
    layer1_weights_21 = tf.Variable(tf.truncated_normal(
        [5, 5, num_channels, depth], stddev = 0.1))
    layer1_biases_21 = tf.Variable(tf.zeros([depth]))
    
    # CONV 3x3 2
    layer2_weights_11 = tf.Variable(tf.truncated_normal(
        [3, 3, depth, depth], stddev = 0.1))
    layer2_biases_11 = tf.Variable(tf.zeros([depth]))    
    
    # CONV 5x5 2
    layer2_weights_21 = tf.Variable(tf.truncated_normal(
        [5, 5, depth, depth], stddev = 0.1))
    layer2_biases_21 = tf.Variable(tf.zeros([depth]))      
    
    
    # fully connected layer1
    # divide by 4 because of (stride 2) x 2
    # turn a 3D into 1D array: flatting
    # num of hidden: hidden neutrons
    ful_layer_len = 896
    layer3_weights = tf.Variable(tf.truncated_normal(
        [ful_layer_len, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # fully connected layer2
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data):
        
        # ConvNet layer 3x3 1
        conv1_11 = tf.nn.conv2d(data, layer1_weights_11 , strides = [1,2,2,1], padding = 'SAME')
        hidden1_11 = tf.nn.relu(conv1_11 + layer1_biases_11 )
        # Max Pooling 2x2 
        pooling1_12 = tf.nn.pool(hidden1_11, window_shape=[3,3],pooling_type='MAX',
                              strides = [2,2], padding = "SAME")
        hidden1_12 = tf.nn.relu(pooling1_12)
        
        # ConvNet layer 5x5 1
        conv1_21 = tf.nn.conv2d(data, layer1_weights_21 , strides = [1,2,2,1], padding = 'SAME')
        hidden1_21 = tf.nn.relu(conv1_21 + layer1_biases_21 )
        # Max Pooling 2x2 
        pooling1_22 = tf.nn.pool(hidden1_21, window_shape=[3,3],pooling_type='MAX',
                              strides = [2,2], padding = "SAME")
        hidden1_22 = tf.nn.relu(pooling1_22)
        
        # Inception 1
        inception1 = tf.concat([hidden1_12, hidden1_22], axis = 1)
        
        
        # ConvNet layer 3x3 2
        conv2_11 = tf.nn.conv2d(inception1, layer2_weights_11 , strides = [1,1,1,1], padding = 'SAME')
        hidden2_11 = tf.nn.relu(conv2_11 + layer2_biases_11 )
        # Max Pooling 2x2 
        pooling2_12 = tf.nn.pool(hidden2_11, window_shape=[3,3],pooling_type='MAX',
                              strides = [2,2], padding = "SAME")
        hidden2_12 = tf.nn.relu(pooling2_12) 
        
        # ConvNet layer 5x5 2
        conv2_21 = tf.nn.conv2d(inception1, layer2_weights_21 , strides = [1,2,2,1], padding = 'SAME')
        hidden2_21 = tf.nn.relu(conv2_21 + layer2_biases_21 )
        # Max Pooling 2x2 
        pooling2_22 = tf.nn.pool(hidden2_21, window_shape=[3,3],pooling_type='MAX',
                              strides = [2,2], padding = "SAME")
        hidden2_22 = tf.nn.relu(pooling2_12) 
        
        # Inception 2
        inception2 = tf.concat([hidden2_12, hidden2_22], axis = 1)        
        
        shape = inception2.get_shape().as_list()
        
        ## 2 fully connect layers
        ## turn a 4D to a 2D matrix, cube to a array
        reshape = tf.reshape(inception2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden3, layer4_weights) + layer4_biases
   
    # Training computation
    logits = model(tf_train_dataset)
    loss = (tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
            + 0.001*(tf.nn.l2_loss(layer1_weights_11) + tf.nn.l2_loss(layer1_biases_11)  
                   + tf.nn.l2_loss(layer1_weights_21) + tf.nn.l2_loss(layer1_biases_21)
                   + tf.nn.l2_loss(layer2_weights_11) + tf.nn.l2_loss(layer2_biases_11) 
                   + tf.nn.l2_loss(layer2_weights_21) + tf.nn.l2_loss(layer2_biases_21)
                   + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) 
                   + tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases) ))
        
    # Learining Rate
    global_step = tf.Variable(0, name= 'global_step', trainable = False)
    learning_rate = tf.train.exponential_decay(0.025, global_step, 1000, 0.95, staircase=True)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [80]:
num_steps = 200000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps+1):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            print('Learinng Rate: %f' % learning_rate.eval())
        if (step % 10000 == 0):
            print('## Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.643815
Minibatch accuracy: 7.8%
Validation accuracy: 9.3%
Learinng Rate: 0.025000
## Test accuracy: 8.9%
Minibatch loss at step 10000: 1.014532
Minibatch accuracy: 85.9%
Validation accuracy: 89.5%
Learinng Rate: 0.014968
## Test accuracy: 94.8%
Minibatch loss at step 20000: 0.905507
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Learinng Rate: 0.008962
## Test accuracy: 95.3%
Minibatch loss at step 30000: 0.761721
Minibatch accuracy: 93.8%
Validation accuracy: 90.6%
Learinng Rate: 0.005366
## Test accuracy: 95.8%
Minibatch loss at step 40000: 0.585580
Minibatch accuracy: 95.3%
Validation accuracy: 91.0%
Learinng Rate: 0.003213
## Test accuracy: 95.9%
Minibatch loss at step 50000: 0.635812
Minibatch accuracy: 93.8%
Validation accuracy: 91.1%
Learinng Rate: 0.001924
## Test accuracy: 96.0%
Minibatch loss at step 60000: 0.555514
Minibatch accuracy: 95.3%
Validation accuracy: 91.0%
Learinng Rate: 0.001152
## Test accuracy: 96.0%
Minibatch loss 

### Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

### Problem 2 


Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.